## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,2021-05-30 16:50:08,22.0752,-159.3190,69.78,88,1,3.40,clear sky
1,1,Cape Town,ZA,2021-05-30 16:45:09,-33.9258,18.4232,61.21,69,0,4.50,clear sky
2,2,Hermanus,ZA,2021-05-30 16:50:08,-34.4187,19.2345,58.89,80,79,3.24,broken clouds
3,3,Khandbari,NP,2021-05-30 16:50:08,27.3747,87.2039,71.85,83,35,1.66,scattered clouds
4,4,Ushuaia,AR,2021-05-30 16:50:09,-54.8000,-68.3000,31.62,100,90,9.22,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df = preferred_cities_df.dropna(axis = 0, how ='any')
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Hithadhoo,MV,2021-05-30 16:50:09,-0.6000,73.0833,83.21,73,52,16.64,broken clouds
8,8,Atuona,PF,2021-05-30 16:50:10,-9.8000,-139.0333,80.33,60,87,8.05,overcast clouds
13,13,Garissa,KE,2021-05-30 16:50:12,-0.4536,39.6401,86.34,42,96,19.71,overcast clouds
20,20,Bandarbeyla,SO,2021-05-30 16:50:15,9.4942,50.8122,79.54,85,22,29.86,few clouds
25,25,Port Maria,JM,2021-05-30 16:45:56,18.3685,-76.8895,85.96,82,7,11.18,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Hithadhoo,MV,2021-05-30 16:50:09,-0.6000,73.0833,83.21,73,52,16.64,broken clouds
8,8,Atuona,PF,2021-05-30 16:50:10,-9.8000,-139.0333,80.33,60,87,8.05,overcast clouds
13,13,Garissa,KE,2021-05-30 16:50:12,-0.4536,39.6401,86.34,42,96,19.71,overcast clouds
20,20,Bandarbeyla,SO,2021-05-30 16:50:15,9.4942,50.8122,79.54,85,22,29.86,few clouds
25,25,Port Maria,JM,2021-05-30 16:45:56,18.3685,-76.8895,85.96,82,7,11.18,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Hithadhoo,MV,83.21,broken clouds,-0.6000,73.0833,
8,Atuona,PF,80.33,overcast clouds,-9.8000,-139.0333,
13,Garissa,KE,86.34,overcast clouds,-0.4536,39.6401,
20,Bandarbeyla,SO,79.54,few clouds,9.4942,50.8122,
25,Port Maria,JM,85.96,clear sky,18.3685,-76.8895,
30,Galle,LK,80.44,overcast clouds,6.0367,80.2170,
32,Quelimane,MZ,75.15,clear sky,-17.8786,36.8883,
34,Georgetown,MY,85.89,few clouds,5.4112,100.3354,
35,Mehndawal,IN,87.89,clear sky,26.9753,83.1117,
36,Butaritari,KI,82.76,light rain,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.  
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
x = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = z.dropna()
clean_hotel_df

NameError: name 'z' is not defined

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
